In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install ultralytics lxml opencv-python easyocr scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 100.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.2/978.2 kB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.6/300.6 kB 23.2 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO

model = YOLO("/content/drive/MyDrive/ANPR_Model_Full/weights/best.pt")
model.info()


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Model summary: 170 layers, 25,856,899 parameters, 0 gradients, 79.1 GFLOPs


(170, 25856899, 0, 79.0656)

In [ ]:
model.predict(
    source="/content/drive/MyDrive/kaggle_datasets/indian_vehicle_dataset/all_images/1a0da3dc-6667-4092-bc2a-a541a06f1e90___3e7fd381-0ae5-4421-8a70-279ee0ec1c61_Nissan-Terrano9.jpg",
    conf=0.4,
    save=True
)



image 1/1 /content/drive/MyDrive/kaggle_datasets/indian_vehicle_dataset/all_images/1a0da3dc-6667-4092-bc2a-a541a06f1e90___3e7fd381-0ae5-4421-8a70-279ee0ec1c61_Nissan-Terrano9.jpg: 640x640 1 number_plate, 1590.8ms
Speed: 8.2ms preprocess, 1590.8ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)
Results saved to /content/runs/detect/predict2


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'number_plate'}
 obb: None
 orig_img: array([[[150, 207, 239],
         [155, 212, 244],
         [158, 215, 247],
         ...,
         [231, 246, 248],
         [226, 241, 243],
         [210, 225, 227]],
 
        [[153, 210, 242],
         [156, 213, 245],
         [157, 214, 246],
         ...,
         [228, 243, 245],
         [223, 238, 240],
         [208, 223, 225]],
 
        [[153, 210, 242],
         [155, 212, 244],
         [154, 211, 243],
         ...,
         [227, 242, 244],
         [220, 235, 237],
         [204, 219, 221]],
 
        ...,
 
        [[153, 157, 152],
         [156, 160, 155],
         [154, 158, 153],
         ...,
         [138, 141, 132],
         [137, 140, 131],
         [134, 137, 128]],
 
        [[153, 157, 152],
         [156, 160, 155],
         [153, 157, 152],
         ...,
         [13

In [ ]:
import cv2
import easyocr
import numpy as np
from ultralytics import YOLO

In [ ]:
# Load trained YOLO model
model = YOLO("/content/drive/MyDrive/ANPR_Model_Full/weights/best.pt")

# Initialize OCR (English only is enough for Indian plates)
reader = easyocr.Reader(['en'])


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [ ]:
def preprocess_plate(plate):
    gray = cv2.cvtColor(plate, cv2.COLOR_BGR2GRAY)
    blur = cv2.bilateralFilter(gray, 11, 17, 17)
    thresh = cv2.adaptiveThreshold(
        blur, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY,
        11, 2
    )
    return thresh

In [ ]:
img_path = "/content/drive/MyDrive/kaggle_datasets/indian_vehicle_dataset/all_images/1a0da3dc-6667-4092-bc2a-a541a06f1e90___3e7fd381-0ae5-4421-8a70-279ee0ec1c61_Nissan-Terrano9.jpg"

img = cv2.imread(img_path)

results = model(img, conf=0.25)

for r in results:
    for box in r.boxes.xyxy:
        x1, y1, x2, y2 = map(int, box)

        # Crop number plate
        plate = img[y1:y2, x1:x2]

        # Preprocess
        plate_prep = preprocess_plate(plate)

        # OCR
        ocr_result = reader.readtext(plate_prep)

        if ocr_result:
            plate_text = ocr_result[0][1]
            print("✅ Detected Plate Number:", plate_text)

        # Draw result
        cv2.rectangle(img, (x1,y1), (x2,y2), (0,255,0), 2)
        cv2.putText(img, plate_text, (x1, y1-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)

# Save output
cv2.imwrite("/content/drive/MyDrive/plate_reading.jpg", img)


0: 640x640 1 number_plate, 1525.8ms
Speed: 8.3ms preprocess, 1525.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
✅ Detected Plate Number: KAOLMN3622


True